In [1]:
# TO DO

# DONE # 1. IMPORT ROBERTA MODEL
# DONE # 2. GET ROBERTA MODEL WORKING ON TEXT
# DONE # 3. GET GUI WORKING
# DONE # 4. CONNECT MODEL TO GUI
# DONE # 5. GET GUI WORKING WITH ROBERTA
# 6. MAKE IT LOOK NICE ENOUGH
# 7. POST TO GITHUB
# 8. UPDATE RESUME

In [2]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter.scrolledtext import ScrolledText
from tkinter import StringVar


In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [5]:
def polarity_scores(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
    'Negative': scores[0],
    'Neutral': scores[1],
    'Positive': scores[2]
    }
    return scores_dict

In [11]:
class LiveSentiment():
    def __init__(self):
        self.root = tk.Tk()
        self.root.title('Live Sentiment')
        self.root.geometry('500x500')
        self.scores = None
        self.sentiment = None

        self.neg = StringVar()
        self.neu = StringVar()
        self.pos = StringVar()
        self.sent = StringVar()

        self.label_neg = tk.Label(self.root, textvariable = self.neg, font=('Arial', 20))
        self.label_neu = tk.Label(self.root, textvariable = self.neu, font=('Arial', 20))
        self.label_pos = tk.Label(self.root, textvariable = self.pos, font=('Arial', 20))
        self.label_sent = tk.Label(self.root, textvariable = self.sent, font=('Arial', 20))

        self.label_neg.pack()
        self.label_neu.pack()
        self.label_pos.pack()
        self.label_sent.pack()

        self.text = ScrolledText(self.root, font=('Arial', 14))
        self.text.bind('<KeyRelease>', self.check)
        self.text.pack()

        self.old_spaces = 0

        self.root.mainloop()

    # CHECK FUNCTION AT EACH SPACE
    def check(self, event):
        content = self.text.get('1.0', tk.END)
        space_count = content.count(' ')
        if space_count != self.old_spaces:
            self.old_spaces = space_count
            
            # CALCULATE POLARITY AND UPDATE 
            self.scores = polarity_scores(content)
            self.neg.set(f'Negative: {np.round(float(self.scores["Negative"]), 2)*100}%')
            self.neu.set(f'Neutral: {np.round(float(self.scores["Neutral"]), 2)*100}%')
            self.pos.set(f'Positive: {np.round(float(self.scores["Positive"]), 2)*100}%')

            self.sent.set(f'Sentiment: {max(self.scores, key=self.scores.get)}')

In [12]:
LiveSentiment()